In [1]:
import tensorflow as tf
import os
import pickle 
import numpy as np

CIFAR_DIR = "./cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [27]:
def load_data(filename):
         with open(filename,'rb') as f:
            data = pickle.load(f,encoding="bytes")
            return data[b'data'],data[b'labels']
        
class CifarData:
    def __init__(self,filenames,need_shuffle):
        all_data = []
        all_labels =[]
        for filename in filenames:
            data,labels= load_data(filename)
            for item,label in zip(data,labels):# 打包为元组的列表 (data,label)
                if label in [0,1]:
                    all_data.append(item)
                    all_labels.append(label)
        self._data = np.vstack(all_data)
        self._data = self._data/127.5-1 #图片像素【0，255】归一化到【-1，1】
        self._labels = np.hstack(all_labels)
        print(self._data.shape)
        print(self._labels.shape)
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
              self._shuffle_data()
                
    def _shuffle_data(self):
        #[0,1,2,3,4,5] ->[5,3,2,4,0,1]
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
        
    def next_batch(self,batch_size):
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator:end_indicator]
        batch_labels = self._labels[self._indicator:end_indicator]
        self._indicator = end_indicator
        return batch_data,batch_labels
train_filenames = [os.path.join(CIFAR_DIR,'data_batch_%d'%i)for i in range(1,6)]
test_filenames  = [os.path.join(CIFAR_DIR,'test_batch')]

train_data = CifarData(train_filenames,True)
test_data = CifarData(test_filenames,False)
# batch_data ,batch_labels  = train_data.next_batch(10)
# print(batch_data)
# print(batch_labels)

(10000, 3072)
(10000,)
(2000, 3072)
(2000,)


In [3]:
x = tf.placeholder(tf.float32,[None,3072])
y = tf.placeholder(tf.int64,[None])

#(3072,,1)
w = tf.get_variable('w',[x.get_shape()[-1],1],
                     initializer = tf.random_normal_initializer(0,1))
# (1,)
b = tf.get_variable('b',[1],
                     initializer = tf.constant_initializer(0.0))

# [None,3072]*[3072,1]=[None,1]

y_ = tf.matmul(x,w) + b
#[None,1]
p_y_1 = tf.nn.sigmoid(y_)
#[None ,1]
y_reshaped  = tf.reshape(y,(-1,1))#转换成一列
y_reshaped_float = tf.cast(y_reshaped,tf.float32)
loss = tf.reduce_mean(tf.square(y_reshaped_float - p_y_1))

predict = p_y_1 > 0.5
#[1,0,0,1,1,1,0,0]
correct_prediction = tf.equal(tf.cast(predict,tf.int64),y_reshaped)
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float64))

with tf.name_scope("train_op"):
        train_op = tf.train.AdamOptimizer(1e-3).minimize(loss);

In [28]:
init  = tf.global_variables_initializer()
batch_size = 20
train_steps = 100000
test_steps = 100

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data,batch_labels = train_data.next_batch(batch_size)
        loss_val,acc_val,_=sess.run([loss,accuracy,train_op],feed_dict={x:batch_data ,y:batch_labels})
        if i %500 == 0:
             print('train step:%d,loss:%4.5f,acc:%4.5f'\
                   %(i,loss_val,acc_val))
        if (i+1)%5000 == 0:
            test_data = CifarData(test_filenames,False)
            all_test_acc_val =[]
            for j in range(test_steps):
                 test_batch_data,test_batch_labels = test_data.next_batch(batch_size)
                 test_acc_val = sess.run(
                     [accuracy],
                    feed_dict={
                           x:test_batch_data,
                           y:test_batch_labels
                                })
                 all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[test]step:%d,acc:%4.5f'%(i+1,test_acc))

train step:0,loss:0.37439,acc:0.60000
train step:500,loss:0.40021,acc:0.60000
train step:1000,loss:0.15007,acc:0.85000
train step:1500,loss:0.18488,acc:0.80000
train step:2000,loss:0.11095,acc:0.90000
train step:2500,loss:0.10774,acc:0.90000
train step:3000,loss:0.24463,acc:0.75000
train step:3500,loss:0.24987,acc:0.75000
train step:4000,loss:0.20767,acc:0.80000
train step:4500,loss:0.14409,acc:0.85000
(2000, 3072)
(2000,)
[test]step:5000,acc:0.80000
train step:5000,loss:0.11858,acc:0.85000
train step:5500,loss:0.15165,acc:0.85000
train step:6000,loss:0.31837,acc:0.65000
train step:6500,loss:0.25018,acc:0.75000
train step:7000,loss:0.35178,acc:0.65000
train step:7500,loss:0.12186,acc:0.85000
train step:8000,loss:0.05000,acc:0.95000
train step:8500,loss:0.12643,acc:0.85000
train step:9000,loss:0.29930,acc:0.70000
train step:9500,loss:0.12181,acc:0.85000
(2000, 3072)
(2000,)
[test]step:10000,acc:0.80450
train step:10000,loss:0.20000,acc:0.80000
train step:10500,loss:0.09900,acc:0.90000
t

train step:88000,loss:0.10099,acc:0.90000
train step:88500,loss:0.00044,acc:1.00000
train step:89000,loss:0.10134,acc:0.90000
train step:89500,loss:0.25002,acc:0.75000
(2000, 3072)
(2000,)
[test]step:90000,acc:0.82050
train step:90000,loss:0.05000,acc:0.95000
train step:90500,loss:0.44943,acc:0.55000
train step:91000,loss:0.29997,acc:0.70000
train step:91500,loss:0.15759,acc:0.85000
train step:92000,loss:0.05002,acc:0.95000
train step:92500,loss:0.05025,acc:0.95000
train step:93000,loss:0.05012,acc:0.95000
train step:93500,loss:0.35425,acc:0.65000
train step:94000,loss:0.25000,acc:0.75000
train step:94500,loss:0.15013,acc:0.85000
(2000, 3072)
(2000,)
[test]step:95000,acc:0.82400
train step:95000,loss:0.15026,acc:0.85000
train step:95500,loss:0.15001,acc:0.85000
train step:96000,loss:0.05385,acc:0.95000
train step:96500,loss:0.15183,acc:0.85000
train step:97000,loss:0.05005,acc:0.95000
train step:97500,loss:0.20648,acc:0.80000
train step:98000,loss:0.15000,acc:0.85000
train step:98500,l